# 1) Veri Keşfi

In [1]:
import urllib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import PIL
import os
from pathlib import Path

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder

from sklearn import set_config; set_config(display='diagram')


## 1.1) Veri Yükleme

**Tüm veri**

WorkinTech'ın mühendislik ekibi 2009'dan beri tüm taksi yolculuğu geçmişini devasa bir Big Query tablosunda `mlops.mlops_all` saklar. 

**Eğitim aşaması verisi**

- Eğitim aşaması `2015-01-01` tarihinde gerçekleşir. Modelimizi eğitmek için bu tarihten sonraki verilere erişemeyiz
- Veri bilimcinin notebook'u burada **200k rastgele örneklenmiş alt küme** üzerinde bir ML modeli eğitir (böylece her şey Veri Bilimcinin laptopunda RAM belleğine sığar)
- WorkinTech'in veri mühendisliği ekibi bu `mlops_all` tablosundan `raw_200k` adlı bir `materialized view` oluşturmuş ve DataScientist'e eğitimi için okuma erişimi vermiştir.

In [ ]:
LOCAL_DATA_PATH = Path('~').joinpath(".workintech", "mlops", "data").expanduser()
GCP_PROJECT = "seraphic-alloy-473611-c7"
BQ_DATASET = "mlops"
DATA_SIZE = "all"  # mlops_all is your BigQuery table
MIN_DATE = '2009-01-01'
MAX_DATE = '2015-01-01'
COLUMN_NAMES_RAW = ('fare_amount',	'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count')

**Aşağıda `GCP_PROJECT`'inizi doldurun 👇**, Daha sonra geçmiş verileri (2015 öncesi) sorgulayalım, tarihe göre sıralanmış (böylece eğitim/test ayrımını kronolojik olarak daha kolay yapabiliriz)!


In [4]:
query = f"""
    SELECT {",".join(COLUMN_NAMES_RAW)}
    FROM `{GCP_PROJECT}`.{BQ_DATASET}.{BQ_DATASET}_{DATA_SIZE}
    WHERE pickup_datetime BETWEEN '{MIN_DATE}' AND '{MAX_DATE}'
    ORDER BY pickup_datetime
    """
print(query)


    SELECT fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
    FROM `data-analytics-469406`.mlops.mlops_all
    WHERE pickup_datetime BETWEEN '2009-01-01' AND '2015-01-01'
    ORDER BY pickup_datetime
    


BigQuery çağrısını yalnızca dosya yerel olarak mevcut değilse yap.  
Aksi halde, her notebook çalıştırmasında BigQuery'yi çağırmaktan kaçınmak için yerel olarak CSV olarak sakla.

In [5]:
data_query_cache_path = Path(LOCAL_DATA_PATH).joinpath("raw", f"query_{MIN_DATE}_{MAX_DATE}_{DATA_SIZE}.csv")

if data_query_cache_path.is_file():
    print("load local file...")
    df = pd.read_csv(data_query_cache_path, parse_dates=['pickup_datetime'])

else:
    print("Querying Big Query server...")
    from google.cloud import bigquery

    client = bigquery.Client(project=GCP_PROJECT)
    query_job = client.query(query)
    result = query_job.result()
    df = result.to_dataframe()

    df.to_csv(data_query_cache_path, header=True, index=False)


Querying Big Query server...


Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/data-analytics-469406/jobs?prettyPrint=false: BigQuery API has not been used in project seraphic-alloy-473611-c7 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/bigquery.googleapis.com/overview?project=seraphic-alloy-473611-c7 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.

Location: None
Job ID: a6100811-5e17-44e1-b37c-fc4bae875dca
 [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'SERVICE_DISABLED', 'domain': 'googleapis.com', 'metadata': {'serviceTitle': 'BigQuery API', 'activationUrl': 'https://console.developers.google.com/apis/api/bigquery.googleapis.com/overview?project=seraphic-alloy-473611-c7', 'containerInfo': 'seraphic-alloy-473611-c7', 'consumer': 'projects/seraphic-alloy-473611-c7', 'service': 'bigquery.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.Local

In [ ]:
df


In [ ]:
df.info()


### 1.1.1) Veri Sıkıştırma

DataFrame'imizi sayısal `dtypes` değerlerini düşürerek sıkıştıralım
- `float64`'den `float32`'ye
- `int64`'den `int8`'e

Bunu yapmak için sütunlarında döngü yaparız ve her biri için [`pd.to_numeric`](https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html) kullanarak `dtypes` değerini mümkün olduğunca düşürürüz

**💡 1) ML Ops Bellek Optimizasyonu `dtype` sıkıştırma hakkında daha fazla bilgi okuyun**

**💡 2) Daha sonra, aşağıdaki kodu anlayın ve çalıştırın**

In [ ]:
def compress(df, **kwargs):
    """
    Reduces the size of the DataFrame by downcasting numerical columns
    """
    input_size = df.memory_usage(index=True).sum()/ 1024**2
    print("old dataframe size: ", round(input_size,2), 'MB')

    in_size = df.memory_usage(index=True).sum()

    for t in ["float", "integer"]:
        l_cols = list(df.select_dtypes(include=t))

        for col in l_cols:
            df[col] = pd.to_numeric(df[col], downcast=t)

    out_size = df.memory_usage(index=True).sum()
    ratio = (1 - round(out_size / in_size, 2)) * 100

    print("optimized size by {} %".format(round(ratio,2)))
    print("new DataFrame size: ", round(out_size / 1024**2,2), " MB")

    return df


In [ ]:
df = compress(df, verbose=True)
df.head(1)


Optimize edilmiş dtypes'ları kontrol edelim

In [ ]:
df.dtypes


💡 Int8 bazı bilimsel kütüphaneler için biraz çok küçük, int16 kullanalım

In [ ]:
df.passenger_count = df.passenger_count.astype("int16")


💡 Next time, we can force dtypes directly **at loading time** as follow:

```python
query_job = client.query(query)
result = query_job.result() 
df = result.to_dataframe(dtypes=DTYPES_RAW)
```

In [ ]:
DTYPES_RAW = {
    "key": "datetime64[ns, UTC]",
    "fare_amount": "float32",
    "pickup_datetime": "datetime64[ns, UTC]",
    "pickup_longitude": "float32",
    "pickup_latitude": "float32",
    "dropoff_longitude": "float32",
    "dropoff_latitude": "float32",
    "passenger_count": "int16"
}


## 1.2) Veri Temizleme

In [ ]:
df.describe()


In [ ]:
df.shape


gereksiz satırları kaldır

In [ ]:
df = df.drop_duplicates()
df.shape


Hatalı işlemleri kaldır

In [ ]:
df = df.dropna(how='any', axis=0)
print(df.shape)
df = df[df.passenger_count > 0]
df = df[df.fare_amount > 0]
print(df.shape)


Coğrafi olarak alakasız işlemleri (satırları) kaldır

In [ ]:
# Let's check NYC bouding boxes
# Load image of NYC map
bounding_boxes = (-74.3, -73.7, 40.5, 40.9)

url = 'https://d37p7d5kaxknzw.cloudfront/projects/nyc.png'
nyc_map = np.array(PIL.Image.open(urllib.request.urlopen(url)))

plt.imshow(nyc_map);


In [ ]:
df = df[df["pickup_latitude"].between(left=40.5, right=40.9)]
df = df[df["dropoff_latitude"].between(left=40.5, right=40.9)]
df = df[df["pickup_longitude"].between(left=-74.3, right=-73.7)]
df = df[df["dropoff_longitude"].between(left=-74.3, right=-73.7)]


In [ ]:
df.describe()


In [ ]:
# Let's cap training set to reasonable values
df = df[df.fare_amount < 400]
df = df[df.passenger_count < 8]


In [ ]:
df


## 1.3) Veri Görselleştirme

In [ ]:
# Plot histogram of fare
df.fare_amount.hist(bins=100, figsize=(14,3))
plt.xlabel('fare $USD')
plt.title('Histogram')


In [ ]:
# this function will be used more often to plot data on the NYC map
def plot_on_map(df, BB, nyc_map, s=10, alpha=0.2):
    fig, axs = plt.subplots(1, 2, figsize=(16,10))

    axs[0].scatter(df.pickup_longitude, df.pickup_latitude, zorder=1, alpha=alpha, c='red', s=s)
    axs[0].set_xlim((BB[0], BB[1]))
    axs[0].set_ylim((BB[2], BB[3]))
    axs[0].set_title('Pickup locations')
    axs[0].imshow(nyc_map, zorder=0, extent=BB)

    axs[1].scatter(df.dropoff_longitude, df.dropoff_latitude, zorder=1, alpha=alpha, c='blue', s=s)
    axs[1].set_xlim((BB[0], BB[1]))
    axs[1].set_ylim((BB[2], BB[3]))
    axs[1].set_title('Dropoff locations')
    axs[1].imshow(nyc_map, zorder=0, extent=BB)


In [ ]:
# Plot training data on map
plot_on_map(df, bounding_boxes, nyc_map, s=1, alpha=0.3)


In [ ]:
plot_on_map(df, bounding_boxes, nyc_map, s=20, alpha=1.0)


In [ ]:
def plot_hires(df, BB, figsize=(12, 12), ax=None, c=('r', 'b')):
    if ax == None:
        fig, ax = plt.subplots(1, 1, figsize=figsize)

    def select_within_boundingbox(df, BB):
        return (df.pickup_longitude >= BB[0]) & (df.pickup_longitude <= BB[1]) & \
            (df.pickup_latitude >= BB[2]) & (df.pickup_latitude <= BB[3]) & \
            (df.dropoff_longitude >= BB[0]) & (df.dropoff_longitude <= BB[1]) & \
            (df.dropoff_latitude >= BB[2]) & (df.dropoff_latitude <= BB[3])

    idx = select_within_boundingbox(df, BB)
    ax.scatter(df[idx].pickup_longitude, df[idx].pickup_latitude, c="red", s=0.01, alpha=0.5)
    ax.scatter(df[idx].dropoff_longitude, df[idx].dropoff_latitude, c="blue", s=0.01, alpha=0.5)


In [ ]:
plot_hires(df, (-74.1, -73.7, 40.6, 40.9))


In [ ]:
plot_hires(df, (-74, -73.95, 40.7, 40.8))


### 1.4) Temel Skor - Ön Sezgiler

Bir temel model en azından en belirgin özelliği dikkate almalıdır: `pickup` ve `dropoff` arasındaki mesafe.

Doğru mesafe metriği uygun şekilde "Manhattan mesafesi" (L1 mesafesi) olarak adlandırılır ve iki nokta arasındaki yatay ve dikey mesafelerin toplamını hesaplar, köşegen (Öklid, L2) mesafesi yerine.

In [ ]:
import math

def manhattan_distance(start_lat: float, start_lon: float, end_lat: float, end_lon: float) -> float:
    """
    Calculate the Manhattan distance between in km two points on the earth (specified in decimal degrees).
    """
    earth_radius = 6371

    lat_1_rad, lon_1_rad = math.radians(start_lat), math.radians(start_lon)
    lat_2_rad, lon_2_rad = math.radians(end_lat), math.radians(end_lon)

    dlon_rad = lon_2_rad - lon_1_rad
    dlat_rad = lat_2_rad - lat_1_rad

    manhattan_rad = abs(dlon_rad) + abs(dlat_rad)
    manhattan_km = manhattan_rad * earth_radius

    return manhattan_km


In [ ]:
%%time
df.apply(lambda row: manhattan_distance(row["pickup_latitude"], row["pickup_longitude"], row["dropoff_latitude"], row["dropoff_longitude"]), axis=1)


☝️ Aşağıdaki kod biraz zaman alır ve optimize edilmemiştir.

Bir DataFrame üzerinde satır satır bir eşleme fonksiyonu uygulamak çok kötü bir mühendislik pratiğidir, çünkü DataFrame'ler bellekte "sütun sütun" saklanır. "Column-major" veri saklama formatları hakkında konuşuyoruz. 

`df.apply(..., axis=1)` bir python `for` döngüsüne eşdeğerdir ve NumPy'nin vektörleştirilmiş işlemlerini kullanmaz.

👇 Bunun yerine kodumuzu vektörleştirelim! Birkaç yüz faktörle iyileşmeyi fark edin 💪

In [ ]:
def manhattan_distance_vectorized(df: pd.DataFrame, start_lat: str, start_lon: str, end_lat: str, end_lon: str) -> dict:
    """
    Calculate the Manhattan distance in km between two points on the earth (specified in decimal degrees).
    Vectorized version for pandas df
    """
    earth_radius = 6371

    lat_1_rad, lon_1_rad = np.radians(df[start_lat]), np.radians(df[start_lon])
    lat_2_rad, lon_2_rad = np.radians(df[end_lat]), np.radians(df[end_lon])

    dlon_rad = lon_2_rad - lon_1_rad
    dlat_rad = lat_2_rad - lat_1_rad

    manhattan_rad = np.abs(dlon_rad) + np.abs(dlat_rad)
    manhattan_km = manhattan_rad * earth_radius

    return manhattan_km


In [ ]:
%%time
manhattan_distance_vectorized(df, "pickup_latitude", "pickup_longitude","dropoff_latitude", "dropoff_longitude")


In [ ]:
df['distance'] = manhattan_distance_vectorized(df, "pickup_latitude", "pickup_longitude","dropoff_latitude", "dropoff_longitude")
df['distance'].hist(bins=50)
plt.title("distance (km)")


Manuel olarak eklediğimiz `distance` sütununu düşürelim ve şimdi gerçek bir ön işleme pipeline'ı oluşturalım

In [ ]:
df = df.drop(columns=['distance'])
df.shape


# 2) Eğitim/Doğrulama/Test Ayrımı

🚨 Zaman damgalı verilerle uğraşıyoruz:
- Eğitim/doğrulama/test ayrımını **kronolojik** bir şekilde yapmak önemlidir.
- Çok uzun doğrulama veya test kümelerini tutmak istemeyiz: gerçek hayatta makroekonomik koşullar hızla değişiyor!
- `split_ratio` ne kadar küçük olursa, modelimizi o kadar sık yeniden eğitmemiz gerekecek
- 6 yıllık bir veri kümesiyle, doğrulama için 1 ay ilerisi, test için 1 ay ilerisi tutmak tamamen uygun
- Üretimde, bu model performansımızın gelecekte 1 aydan fazla uzanmasına güvenmememiz gerektiği anlamına gelir

In [ ]:
split_ratio = 0.02 # ~1 month for val, ~1 month for test

test_length = int(len(df) * split_ratio)
val_length = int((len(df)-test_length) * split_ratio)
train_length = len(df) - val_length - test_length

df_train = df.iloc[:train_length, :].sample(frac=1) # Shuffle datasets to improve training
df_val = df.iloc[train_length: train_length + val_length, :].sample(frac=1)
df_test = df.iloc[train_length+val_length:, :].sample(frac=1)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

assert len(df_train) + len(df_val) + len(df_test) == len(df)


In [ ]:
print(df_train.pickup_datetime.min())
print(df_train.pickup_datetime.max())
print('---')
print(df_val.pickup_datetime.min())
print(df_val.pickup_datetime.max())
print('---')
print(df_test.pickup_datetime.min())
print(df_test.pickup_datetime.max())


In [ ]:

X = df.drop("fare_amount", axis=1)
y = df[["fare_amount"]]

X_train = df_train.drop("fare_amount", axis=1)
y_train = df_train[["fare_amount"]]

X_val = df_val.drop("fare_amount", axis=1)
y_val = df_val[["fare_amount"]]

X_test = df_test.drop("fare_amount", axis=1)
y_test = df_test[["fare_amount"]]


### Basit Temel: $price = a * distance + b $

In [ ]:
distances_train = np.array(manhattan_distance_vectorized(X_train, "pickup_latitude", "pickup_longitude","dropoff_latitude", "dropoff_longitude"))
distances_val = np.array(manhattan_distance_vectorized(X_val, "pickup_latitude", "pickup_longitude","dropoff_latitude", "dropoff_longitude"))
distances_test = np.array(manhattan_distance_vectorized(X_test, "pickup_latitude", "pickup_longitude","dropoff_latitude", "dropoff_longitude"))


In [ ]:
sns.regplot(x=distances_train, y=y_train);


In [ ]:
from scipy.stats import pearsonr

pearson, p_value = pearsonr(distances_train, y_train)
print(f'{pearson=}')
print(f'{p_value=}')


In [ ]:
from sklearn.linear_model import LinearRegression

baseline_model = LinearRegression()
baseline_model.fit(distances_train[:, None], y_train)


In [ ]:
baseline_pred_val = baseline_model.predict(distances_val[:, None])
baseline_pred_test = baseline_model.predict(distances_test[:, None])
baseline_mae_val = np.mean(np.abs(baseline_pred_val - y_val), axis=0)
baseline_mae_test = np.mean(np.abs(baseline_pred_test - y_test), axis=0)

print(f'mean taxifare prices on train set = {round(float(np.mean(y_train, axis=0)),2)} $')
print(f'🎯 baseline MAE on val set = {round(float(baseline_mae_val),2)} $')
print(f'🎯 baseline MAE on test set = {round(float(baseline_mae_test),2)} $')


# 3) Ön İşleme Pipeline'ı

Bize yalnızca 5 özellik (yolcular + boylam/enlem) ve potansiyel olarak on milyonlarca satır içeren bir veri kümesi veriliyor.

👉 "günün saati" gibi bir sürü "mühendislik" özelliği oluşturmak tamamen mantıklı  
- Yüzlercesi hiçbir problem yaratmaz çünkü büyük sayıda satır modelimizin bu çoklu özelliklerle ilişkili tüm ağırlıkları öğrenmesini sağlayacaktır
- Yoğun, Derin Öğrenme ağı böyle bir durum için çok uygundur

❗️ Aşağıda önerilen ön işlemci **sabit sayıda özellik** (65) çıkarır ve bu **eğitim kümesinden bağımsızdır**. Bunu yüz milyonlarca satıra ölçeklendirirken işe yarayacağını göreceksiniz

## 3.1) Yolcu Ön İşlemcileri

Yolcu sayılarını analiz edelim

In [ ]:
df.passenger_count


In [ ]:
sns.histplot(data=df, x="passenger_count");


In [ ]:
# PASSENGER PIPE
p_min = 1.
p_max = 8.
passenger_pipe = FunctionTransformer(lambda p: (p-p_min)/(p_max-p_min))


In [ ]:
preprocessor = ColumnTransformer(
    [
        ("passenger_scaler", passenger_pipe, ["passenger_count"]),
    ],
)
preprocessor


In [ ]:
pd.DataFrame(preprocessor.fit_transform(X_train))


## 3.2) Zaman Ön İşlemcisi

`pickup_datetime`'dan ilginç öznitelikleri çıkaralım
- günün saati
- haftanın günü
- yılın ayı
- 2009'dan beri gün sayısı (enflasyon parametrelerini kodlayabilir)

In [ ]:
import math

def transform_time_features(X: pd.DataFrame) -> np.ndarray:
    assert isinstance(X, pd.DataFrame)

    timedelta = (X["pickup_datetime"] - pd.Timestamp('2009-01-01T00:00:00', tz='UTC'))/pd.Timedelta(1,'D')

    pickup_dt = X["pickup_datetime"].dt.tz_convert("America/New_York").dt
    dow = pickup_dt.weekday
    hour = pickup_dt.hour
    month = pickup_dt.month

    hour_sin = np.sin(2 * math.pi / 24 * hour)
    hour_cos = np.cos(2*math.pi / 24 * hour)

    return np.stack([hour_sin, hour_cos, dow, month, timedelta], axis=1)

X_time_processed = transform_time_features(X[["pickup_datetime"]])

pd.DataFrame(X_time_processed, columns=["hour_sin", "hour_cos", "dow", "month", "timedelta"]).head()


Daha sonra, tüm 24*7 kategori kombinasyonlarını `X_processed`'da her zaman mevcut olmaya zorlayarak `["haftanın günü", "ay"]`'ı one-hot kodlarız 

(sonunda `X_processed` için sabit bir boyut istediğimizi unutmayın)

In [ ]:
time_categories = [
        np.arange(0, 7, 1),  # days of the week from 0 to 6
        np.arange(1, 13, 1)  # months of the year from 1 to 12
    ]

OneHotEncoder(categories=time_categories, sparse_output=False)\
    .fit_transform(X_time_processed[:,[2,3]]) # column index [2,3] for ['dow', 'month'] !


Ve bunu `timedelta` sütununun bir tür "Min-Max" yeniden ölçeklemesi ile birleştirin

In [ ]:
print(X_time_processed[:,-1].min())
print(X_time_processed[:,-1].max())


In [ ]:
timedelta_min = 0
timedelta_max = 2190 # Our model may extend in the future. No big deal if the scaled data extend slightly beyond 1.0


In [ ]:
time_pipe = make_pipeline(
    FunctionTransformer(transform_time_features),
    make_column_transformer(
        (OneHotEncoder(
            categories=time_categories,
            sparse_output=False,
            handle_unknown="ignore"
        ), [2, 3]), # corresponds to columns ["day of week", "month"], not the other columns

        (FunctionTransformer(lambda year: (year - timedelta_min) / (timedelta_max - timedelta_min)), [4]), # min-max scale the columns 4 ["year"]
        remainder="passthrough" # keep hour_sin and hour_cos
    )
)

preprocessor = ColumnTransformer(
    [
        ("passenger_scaler", passenger_pipe, ["passenger_count"]),
        ("time_preproc", time_pipe, ["pickup_datetime"]),
    ],
)

preprocessor


In [ ]:
pd.DataFrame(preprocessor.fit_transform(X_train)).plot(kind='box');


☝️ Yaklaşık olarak ortalanmış ve ölçeklenmiş 23 özellik

## 3.3) Mesafe Pipeline'ı

Hem haversine hem de Manhattan mesafelerini özellik olarak ekleyelim

In [ ]:
lonlat_features = ["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"]


In [ ]:
def distances_vectorized(df: pd.DataFrame, start_lat: str, start_lon: str, end_lat: str, end_lon: str) -> dict:
    """
    Calculate the haversine and Manhattan distances between two points (specified in decimal degrees).
    Vectorized version for pandas df
    Computes distance in Km
    """
    earth_radius = 6371

    lat_1_rad, lon_1_rad = np.radians(df[start_lat]), np.radians(df[start_lon])
    lat_2_rad, lon_2_rad = np.radians(df[end_lat]), np.radians(df[end_lon])

    dlon_rad = lon_2_rad - lon_1_rad
    dlat_rad = lat_2_rad - lat_1_rad

    manhattan_rad = np.abs(dlon_rad) + np.abs(dlat_rad)
    manhattan_km = manhattan_rad * earth_radius

    a = (np.sin(dlat_rad / 2.0)**2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon_rad / 2.0)**2)
    haversine_rad = 2 * np.arcsin(np.sqrt(a))
    haversine_km = haversine_rad * earth_radius

    return dict(
        haversine = haversine_km,
        manhattan = manhattan_km
    )


In [ ]:
def transform_lonlat_features(X:pd.DataFrame)-> pd.DataFrame:
    assert isinstance(X, pd.DataFrame)
    res = distances_vectorized(X, *lonlat_features)

    return pd.DataFrame(res)

distances = transform_lonlat_features(X[lonlat_features])
distances


In [ ]:
dist_min = 0
dist_max = 100


In [ ]:
distance_pipe = make_pipeline(
    FunctionTransformer(transform_lonlat_features),
    FunctionTransformer(lambda dist: (dist - dist_min) / (dist_max - dist_min))
    )
distance_pipe


In [ ]:
preprocessor = ColumnTransformer(
    [
        ("passenger_scaler", passenger_pipe, ["passenger_count"]),
        ("time_preproc", time_pipe, ["pickup_datetime"]),
        ("dist_preproc", distance_pipe, lonlat_features),
    ],
)
preprocessor


In [ ]:
X_processed = pd.DataFrame(preprocessor.fit_transform(X_train))
X_processed.plot(kind='box');


☝️ Yaklaşık olarak ölçeklenmiş 25 özellik

## 3.4) GeoHasher

Son olarak, **bölgeler** hakkında bilgi ekleyelim! 

Bazıları diğerlerinden daha pahalı olabilir (örn. JFK havalimanı!)

Jeouzaysal bilgiyi _bucketize_ etmek için `pygeohash` kullanacağız

In [ ]:
import pygeohash as gh


💡 pygeohash (lat,lon)'u seçilen kesinliklerde jeospacial "kare kovalar"a dönüştürür. Ne kadar çok kesinlik isterseniz, o kadar çok "kova" olasılığı vardır!

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/07-ML-OPS/geohashes.png">

In [ ]:
x0 = X_train.iloc[0,:]
(x0.pickup_latitude, x0.pickup_longitude)


In [ ]:
print(gh.encode(x0.pickup_latitude, x0.pickup_longitude, precision=3))
print(gh.encode(x0.pickup_latitude, x0.pickup_longitude, precision=4))
print(gh.encode(x0.pickup_latitude, x0.pickup_longitude, precision=5))


👇 Bunu TÜM veri setumuze uygulayalım (bu ön işlemenin çok uzun zaman alabileceğini unutmayın!)

In [ ]:
def compute_geohash(X:pd.DataFrame, precision:int = 5) -> np.ndarray:
    """
    Add a geohash (ex: "dr5rx") of len "precision" = 5 by default
    corresponding to each (lon, lat) tuple, for pick-up and drop-off
    """
    assert isinstance(X, pd.DataFrame)

    X["geohash_pickup"] = X.apply(
        lambda x: gh.encode(x.pickup_latitude, x.pickup_longitude, precision=precision),
        axis=1
    )
    X["geohash_dropoff"] = X.apply(
        lambda x: gh.encode(x.dropoff_latitude, x.dropoff_longitude, precision=precision),
        axis=1
    )

    return X[["geohash_pickup", "geohash_dropoff"]]


In [ ]:
compute_geohash(X_train)


☝️ Bu sefer `pygeohash`'i `df.apply(axis=1)` ile satır satır uygulamaktan başka seçeneğimiz olmadığını ve hesaplamanın biraz zaman aldığını fark edin.

Bu, her zaman vektörleştirilmemiş harici bir Python kütüphanesine güvenmenin tehlikesidir.

👇 En yaygın bölgeler hangileri?

In [ ]:
all_geohashes = pd.concat([
    X_train.apply(lambda x: gh.encode(x.pickup_latitude, x.pickup_longitude, precision=4), axis=1),
    X_train.apply(lambda x: gh.encode(x.dropoff_latitude, x.dropoff_longitude, precision=4), axis=1),
])


In [ ]:
print(len(all_geohashes.value_counts()))
plt.figure(figsize=(15,5))
plt.plot(np.cumsum(all_geohashes.value_counts()[:20])/(2*len(X_train))*100)
plt.title("percentage of taxi rides from/to these districts");


☝️ Sadece ilk 20 bölge önemli. Bunları one-hot kodlayabiliriz.

In [ ]:
most_important_geohash_districts = np.array(all_geohashes.value_counts()[:20].index)
most_important_geohash_districts


In [ ]:
# Let's hard-code below the 20 most frequent district GeoHashes of precision 5,
# covering about 99% of all dropoff/pickup locations.
most_important_geohash_districts = [
    "dr5ru", "dr5rs", "dr5rv", "dr72h", "dr72j", "dr5re", "dr5rk",
    "dr5rz", "dr5ry", "dr5rt", "dr5rg", "dr5x1", "dr5x0", "dr72m",
    "dr5rm", "dr5rx", "dr5x2", "dr5rw", "dr5rh", "dr5x8"
]


Her GeoHash'i yukarıda listelenen ilk 20 farklı kovadan birinde one-hot kodlayalım

In [ ]:
geohash_categories = [
    most_important_geohash_districts,  # pickup district list
    most_important_geohash_districts  # dropoff district list
]

geohash_pipe = make_pipeline(
    FunctionTransformer(compute_geohash),
    OneHotEncoder(
        categories=geohash_categories,
        handle_unknown="ignore",
        sparse_output=False
    )
)
geohash_pipe


## 3.5) Tam Ön İşleme Pipeline'ı

Son ön işlemcimizi özetleyelim

### a) Kodlayıcılar

In [ ]:
import math
import numpy as np
import pandas as pd
import pygeohash as gh


In [ ]:
def transform_time_features(X: pd.DataFrame) -> np.ndarray:
    assert isinstance(X, pd.DataFrame)

    timedelta = (X["pickup_datetime"] - pd.Timestamp('2009-01-01T00:00:00', tz='UTC'))/pd.Timedelta(1,'D')

    pickup_dt = X["pickup_datetime"].dt.tz_convert("America/New_York").dt
    dow = pickup_dt.weekday
    hour = pickup_dt.hour
    month = pickup_dt.month

    hour_sin = np.sin(2 * math.pi / 24 * hour)
    hour_cos = np.cos(2*math.pi / 24 * hour)

    return np.stack([hour_sin, hour_cos, dow, month, timedelta], axis=1)


In [ ]:
def transform_lonlat_features(X: pd.DataFrame) -> pd.DataFrame:

    assert isinstance(X, pd.DataFrame)
    lonlat_features = ["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"]

    def distances_vectorized(df: pd.DataFrame, start_lat: str, start_lon: str, end_lat: str, end_lon: str) -> dict:
        """
        Calculate the haversine and Manhattan distances between two points on the earth (specified in decimal degrees).
        Vectorized version for pandas df
        Computes distance in Km
        """
        earth_radius = 6371

        lat_1_rad, lon_1_rad = np.radians(df[start_lat]), np.radians(df[start_lon])
        lat_2_rad, lon_2_rad = np.radians(df[end_lat]), np.radians(df[end_lon])

        dlon_rad = lon_2_rad - lon_1_rad
        dlat_rad = lat_2_rad - lat_1_rad

        manhattan_rad = np.abs(dlon_rad) + np.abs(dlat_rad)
        manhattan_km = manhattan_rad * earth_radius

        a = (np.sin(dlat_rad / 2.0)**2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon_rad / 2.0)**2)
        haversine_rad = 2 * np.arcsin(np.sqrt(a))
        haversine_km = haversine_rad * earth_radius

        return dict(
            haversine=haversine_km,
            manhattan=manhattan_km)

    result = pd.DataFrame(distances_vectorized(X, *lonlat_features))

    return result


In [ ]:
def compute_geohash(X: pd.DataFrame, precision: int = 5) -> np.ndarray:
    """
    Add a GeoHash (ex: "dr5rx") of len "precision" = 5 by default
    corresponding to each (lon, lat) tuple, for pick-up, and drop-off
    """
    assert isinstance(X, pd.DataFrame)

    X["geohash_pickup"] = X.apply(
        lambda x: gh.encode(x.pickup_latitude, x.pickup_longitude, precision=precision),
        axis=1
    )
    X["geohash_dropoff"] = X.apply(
        lambda x: gh.encode(x.dropoff_latitude, x.dropoff_longitude, precision=precision),
        axis=1
    )

    return X[["geohash_pickup", "geohash_dropoff"]]


### b) Pipeline

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer


In [ ]:
# PASSENGER PIPE
p_min = 1
p_max = 8
passenger_pipe = FunctionTransformer(lambda p: (p - p_min) / (p_max - p_min))

# DISTANCE PIPE
dist_min = 0
dist_max = 100

distance_pipe = make_pipeline(
    FunctionTransformer(transform_lonlat_features),
    FunctionTransformer(lambda dist: (dist - dist_min) / (dist_max - dist_min))
)

# TIME PIPE
timedelta_min = 0
timedelta_max = 2090

time_categories = [
    np.arange(0, 7, 1),  # days of the week
    np.arange(1, 13, 1)  # months of the year
]

time_pipe = make_pipeline(
    FunctionTransformer(transform_time_features),
    make_column_transformer(
        (OneHotEncoder(
            categories=time_categories,
            sparse_output=False,
            handle_unknown="ignore"
        ), [2,3]), # corresponds to columns ["day of week", "month"], not the other columns

        (FunctionTransformer(lambda year: (year - timedelta_min) / (timedelta_max - timedelta_min)), [4]), # min-max scale the columns 4 ["timedelta"]
        remainder="passthrough" # keep hour_sin and hour_cos
    )
)

# GEOHASH PIPE
lonlat_features = [
    "pickup_latitude", "pickup_longitude", "dropoff_latitude",
    "dropoff_longitude"
]

# Below are the 20 most frequent district geohashes of precision 5,
# covering about 99% of all dropoff/pickup locations,
# according to prior analysis in a separate notebook
most_important_geohash_districts = [
    "dr5ru", "dr5rs", "dr5rv", "dr72h", "dr72j", "dr5re", "dr5rk",
    "dr5rz", "dr5ry", "dr5rt", "dr5rg", "dr5x1", "dr5x0", "dr72m",
    "dr5rm", "dr5rx", "dr5x2", "dr5rw", "dr5rh", "dr5x8"
]

geohash_categories = [
    most_important_geohash_districts,  # pickup district list
    most_important_geohash_districts  # dropoff district list
]

geohash_pipe = make_pipeline(
    FunctionTransformer(compute_geohash),
    OneHotEncoder(
        categories=geohash_categories,
        handle_unknown="ignore",
        sparse_output=False
    )
)

# COMBINED PREPROCESSOR
final_preprocessor = ColumnTransformer(
    [
        ("passenger_scaler", passenger_pipe, ["passenger_count"]),
        ("time_preproc", time_pipe, ["pickup_datetime"]),
        ("dist_preproc", distance_pipe, lonlat_features),
        ("geohash", geohash_pipe, lonlat_features),
    ],
    n_jobs=-1,
)


In [ ]:
final_preprocessor


In [ ]:
final_preprocessor.fit(X_train)

X_train_processed = final_preprocessor.transform(X_train)
X_val_processed = final_preprocessor.transform(X_val)
X_test_processed = final_preprocessor.transform(X_test)


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
pd.DataFrame(X_train_processed).plot(kind='box', ax=ax);


In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
sns.heatmap(pd.DataFrame(X_train_processed).corr(), vmin=-1, cmap='RdBu');


Sonuç olarak, çok fazla performans kaybetmeden işlenmiş verimizi muhtemelen float32'ye de sıkıştırabiliriz

In [ ]:
X_train_processed.dtype


In [ ]:
print(X_train_processed.nbytes / 1024**2, "MB")


In [ ]:
# Compress the data a bit
X_train_processed = X_train_processed.astype(np.float32)
X_val_processed = X_val_processed.astype(np.float32)
X_test_processed = X_test_processed.astype(np.float32)

print(X_train_processed.nbytes / 1024**2, "MB")


In [ ]:
pd.DataFrame(X_train_processed)


In [ ]:
pd.DataFrame(X_train_processed).describe().mean(axis=1)


☝️ Ön işlemci eğitim kümesinden bağımsız **sabit** sayıda özellik (65) çıkarır. 

☝️ Ön işlemci aynı zamanda **durumsuz**dur (yani `.fit()` metodu yoktur, yalnızca `.transform()` vardır). Bu, standart ölçeklemenin aksine, "X_train standart sapmaları"nı iç durumlar olarak saklamak zorunda olan saf bir fonksiyon $f:X \rightarrow X_{processed}$ olarak görülebilir.

Bu iki özellik, ML Mühendisliği ekibinin ön işlemeyi yüzlerce GB'lara ölçeklendirmek için işlerini çok daha kolay hale getirecektir.

# 4) Model

## 4.1) Mimari

In [ ]:
from tensorflow import keras
from keras import Model, Sequential, layers, regularizers
from keras.callbacks import EarlyStopping


In [ ]:
def initialize_model(input_shape:tuple) -> Model:
    """
    Initialize the Neural Network with random weights
    """

    reg = regularizers.l1_l2(l1=0.005)

    model = Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(100, activation="relu", kernel_regularizer=reg))
    model.add(layers.BatchNormalization(momentum=0.9))
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(50, activation="relu"))
    model.add(layers.BatchNormalization(momentum=0.9))  # use momentum=0 to only use statistic of the last seen minibatch in inference mode ("short memory"). Use 1 to average statistics of all seen batch during training histories.
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(1, activation="linear"))

    print("✅ model initialized")

    return model


In [ ]:
model = initialize_model(input_shape=X_train_processed.shape[1:])
model.summary()


In [ ]:
learning_rate = 0.0005
batch_size = 256

optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["mae"])


In [ ]:
es = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True,
    verbose=0
)

history = model.fit(
    X_train_processed,
    y_train,
    validation_data=(X_val_processed, y_val),
    epochs=100,
    batch_size=batch_size,
    callbacks=[es],
    verbose=1
)


## 4.2) Performans değerlendirmesi

In [ ]:
print("MAE val", round(model.evaluate(X_val_processed, y_val)[1],2), ' $')
print("MAE test", round(model.evaluate(X_test_processed, y_test)[1],2), ' $')
print("MAE test baseline", round(float(baseline_mae_test),2), ' $')


In [ ]:
y_pred = model.predict(X_test_processed)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15,8))

plt.hist(y_pred, label='pred', color='r', bins=200, alpha=0.3)
plt.hist(y_test, label='truth', color='b', bins=200, alpha=0.3)

plt.legend()
plt.xlim((0,60))


In [ ]:
residuals = y_pred - y_test


In [ ]:
residuals.describe()


In [ ]:
residuals = y_pred - y_test
sns.histplot(residuals)
plt.xlim(xmin=-20, xmax=20)


In [ ]:
residuals.sort_values(by='fare_amount')


In [ ]:
# Residual vs. actual scatter plot
plt.figure(figsize=(15,5))
plt.scatter(x=y_test,y=residuals, alpha=0.1)
plt.xlabel('actual')
plt.ylabel('residuals')


In [ ]:
# Residual vs. predicted scatter plot
plt.figure(figsize=(15,5))
plt.scatter(x=y_pred,y=residuals, alpha=0.1)
plt.xlabel('predicted')
plt.ylabel('residuals')


☝️ Sınırlı 60k veri seti üzerinde eğitilen modelimizin ortalama yolculuk fiyatı 11$'a kıyasla yolculuk başına yaklaşık 2.5$ MAE'si var.  

Basit bir doğrusal regresyon bize yaklaşık 3$ MAE verir, ama şeytan ayrıntılarda gizlidir!

Özellikle, çok uzun/pahalı yolculukları tahmin etmede o kadar iyi değiliz.

# 🧪 Anlayışınızı Test Edin

❓ Arkadaşınızla bu soruları cevaplamaya çalışın
1. Modelin performansından memnun musunuz? 
2. Geliştirmek için herhangi bir fikirler?
3. Durumsuz pipeline nedir (durumlu olanın aksine)?
4. OHEncoder sabit sütun kategorileri ile nasıl çalışır?
5. Sinir Ağında veri normalleştirme nasıl yapılır?

<details>
  <summary markdown='span'>💡 Cevaplar</summary>

1. Doğrusal regresyon temeline göre %25 iyileşmemiz var (ama emin olmak için bunu çapraz doğrulamamız gerekir). Ayrıca, fiyatları trafiğe de bağlı olan taksi yolculuklarında 2$'lık tahmin hatası indirgenemez hata oranına yakın görünüyor.


2. Daha fazla eğitim verisi ekleyerek modeli geliştirebiliriz (bunu daha sonra test edeceğiz). Başka bir umut verici fikir kategorik özelliklerimizin gömülmesini *öğrenmek* olabilir (onları one-hot kodlamak yerine).


3. Durumsuz pipeline gerçek bir `.fit()` metoduna sahip değildir, yalnızca `.transform()` vardır. 


4. Durumsuz olmak için, one-hot kodlanacak `kategorileri` sabit kodladık `OneHotEncoder(categories=categories,...)` ve ölçekleyicilerimizde her sütunun istatistiksel özelliklerini sabit kodladık:  `FunctionTransformer(lambda dist: (dist - dist_min)/(dist_max - dist_min))`


5. TensorFlow modelinde, her yoğun katman arasına eklediğimiz `layers.BatchNormalization()`'ı fark edin, bu da verileri batch-per-batch normalleştirir! Geri yayılım sırasında gradyan kaybolmasını düzeltmeye yardımcı olan harika bir özelliktir!


</details>

❓ Aşağıdaki bu yeni yolculuk `X_new` için fiyat tahmin edin ve sonucu `y_new` olarak `np.ndarray` şeklinde saklayın

In [ ]:
X_train.iloc[0:2,:]['pickup_datetime']


In [ ]:
X_new = pd.DataFrame(dict(
    pickup_datetime=[pd.Timestamp("2013-07-06 17:18:00", tz='UTC')],
    pickup_longitude=[-73.950655],
    pickup_latitude=[40.783282],
    dropoff_longitude=[-73.984365],
    dropoff_latitude=[40.769802],
    passenger_count=[1],
))

X_new


In [ ]:
# Compute y_new
pass  # YOUR CODE HERE


In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'notebook',
    subdir='train_at_scale',
    y_new=y_new
)

result.write()
print(result.check())


Son olarak, Kitt'in ilerlemenizi takip edebilmesi için `make test_kitt` çalıştıralım

In [ ]:
! cd .. && make test_kitt
